In [10]:
%pip install llama-index-readers-file pymupdf
%pip install llama-index-vector-stores-postgres
%pip install llama-index-embeddings-huggingface


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
%pip install llama-index-llms-llama-cpp

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
# sentence transformers
from llama_index.embeddings.huggingface import HuggingFaceEmbedding


embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

In [13]:
#Llama CPP, in this notebook, we use the llama-2-chat-13b-ggml model, along with the proper prompt formatting.

!pip install llama-cpp-python

In [14]:
%pip install llama-index-embeddings-huggingface
%pip install llama-index-llms-llama-cpp

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from llama_index.llms.llama_cpp import LlamaCPP

#model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"
#model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"
model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"


llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    #model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path= 'models\llama-2-7b-chat.Q2_K.gguf',
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    # transform inputs into Llama2 format

    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from models\llama-2-7b-chat.Q2_K.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32           

In [16]:
# init Postgres 
# Using an existing Postgres running at the localhost, create the database we will be using
!pip install psycopg2-binary pgvector asycpg "sqlalchemy[asyncio]" greenlet


  Obtaining dependency information for psycopg2-binary from https://files.pythonhosted.org/packages/25/1f/7ae31759142999a8d06b3e250c1346c4abcdcada8fa884376775dc1de686/psycopg2_binary-2.9.9-cp311-cp311-win_amd64.whl.metadata
  Using cached psycopg2_binary-2.9.9-cp311-cp311-win_amd64.whl.metadata (4.6 kB)
  Obtaining dependency information for pgvector from https://files.pythonhosted.org/packages/20/ad/a5c6c0a5bab412176a0d5e1ec481610e134111b06ee681f3b4407dc17bc5/pgvector-0.3.2-py2.py3-none-any.whl.metadata
  Using cached pgvector-0.3.2-py2.py3-none-any.whl.metadata (12 kB)


ERROR: Could not find a version that satisfies the requirement asycpg (from versions: none)
ERROR: No matching distribution found for asycpg


In [30]:
import psycopg2

dbname = "vector_db"
host = "localhost"
password = 'geniusraver27'
port = "5432"
user = "postgres"
# conn = psycopg2.connect(connection_string)
conn = psycopg2.connect(
    dbname=dbname, 
    user=user, 
    password=password, 
    host=host, 
    port=port)
conn.autocommit = True



OperationalError: connection to server at "localhost" (::1), port 5432 failed: FATAL:  password authentication failed for user "sithu"


In [18]:
from sqlalchemy import make_url
from llama_index.vector_stores.postgres import PGVectorStore

vector_store = PGVectorStore.from_params(
    database="postgres",
    host=host,
    password=password,
    port=port, 
    user=user,
    table_name="llama2_paper",
    embed_dim=384, # Openai embedding dimension
)

In [19]:
# Build an Ingestion Pipeline from Scratch
# 1. Load Data
!mkdir data
!wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "data/llama2.pdf"

A subdirectory or file data already exists.
'wget' is not recognized as an internal or external command,
operable program or batch file.


In [20]:
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader

In [21]:
loader = PyMuPDFReader()
documents = loader.load(file_path=Path("data/llama2.pdf"))

In [22]:
# 2. Use a Text Splitter to split the documents
from llama_index.core.node_parser import SentenceSplitter


In [23]:
text_parser = SentenceSplitter(
    chunk_size=1024,
    # separator = "",
)


In [24]:
text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)

doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

In [25]:
# 3. Manually Construct Nodes from Text Chunks
from llama_index.core.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc = documents[doc_idxs[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)

In [26]:
#4. Generate Embeddings for each Node
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

In [29]:
# Load Nodes into a Vector Store
vector_store.add(nodes)

NotSupportedError: (psycopg2.errors.FeatureNotSupported) extension "vector" is not available
DETAIL:  Could not open extension control file "C:/Program Files/PostgreSQL/16rc1/share/extension/vector.control": No such file or directory.
HINT:  The extension must first be installed on the system where PostgreSQL is running.

[SQL: CREATE EXTENSION IF NOT EXISTS vector]
(Background on this error at: https://sqlalche.me/e/20/tw8g)

In [ ]:
# Build Retrieval Pipeline from Scratch
query_str = "Can you tell me about the key concepts for safety finetuning"

In [ ]:
#1. Genearate a Query Embedding
query_embedding = embed_model.get_query_embedding(query_str)

In [ ]:
#2.Query the Vector Database
# construct vector store query
from llama_index.core.vector_stores import VectorStoreQuery

query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"

vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)


In [ ]:
# returns a VectorStoreQueryResult
query_result = vector_store.query(vector_store_query)
print(query_result.nodes[0].get_content())


In [ ]:
# 3 Parse Result into a Set of Nodes
from llama_index.core.schema import NodeWithScore
from typing import Optional

nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
    nodes_with_scores.append(NodeWithScore(node=node, score=score))

In [ ]:
# Put into a Retirever
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetirever
from typing import Any, List

class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store: PGVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [ ]:
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=2
)

In [ ]:
# Plug this into RetrieverQueryEngine to synthesize a response

from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)

In [ ]:
query_str = "How does Llama 2 perform compared to other open-source models?"

response = query_engine.query(query_str)

In [ ]:
print(str(response))

In [ ]:
print(response.source_nodes[0].get_content())
